# Welcome to Week 2!

## Frontier Model APIs

In Week 1, we used multiple Frontier LLMs through their Chat UI, and we connected with the OpenAI's API.

Today we'll connect with the APIs for Anthropic and Google, as well as OpenAI.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Important Note - Please read me</h2>
            <span style="color:#900;">I'm continually improving these labs, adding more examples and exercises.
            At the start of each week, it's worth checking you have the latest code.<br/>
            First do a <a href="https://chatgpt.com/share/6734e705-3270-8012-a074-421661af6ba9">git pull and merge your changes as needed</a>. Any problems? Try asking ChatGPT to clarify how to merge - or contact me!<br/><br/>
            After you've pulled the code, from the llm_engineering directory, in an Anaconda prompt (PC) or Terminal (Mac), run:<br/>
            <code>conda env update --f environment.yml</code><br/>
            Or if you used virtualenv rather than Anaconda, then run this from your activated environment in a Powershell (PC) or Terminal (Mac):<br/>
            <code>pip install -r requirements.txt</code>
            <br/>Then restart the kernel (Kernel menu >> Restart Kernel and Clear Outputs Of All Cells) to pick up the changes.
            </span>
        </td>
    </tr>
</table>
<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Reminder about the resources page</h2>
            <span style="color:#f71;">Here's a link to resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

## Setting up your keys

If you haven't done so already, you could now create API keys for Anthropic and Google in addition to OpenAI.

**Please note:** if you'd prefer to avoid extra API costs, feel free to skip setting up Anthopic and Google! You can see me do it, and focus on OpenAI for the course. You could also substitute Anthropic and/or Google for Ollama, using the exercise you did in week 1.

For OpenAI, visit https://openai.com/api/  
For Anthropic, visit https://console.anthropic.com/  
For Google, visit https://ai.google.dev/gemini-api  

When you get your API keys, you need to set them as environment variables by adding them to your `.env` file.

```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
```

Afterwards, you may need to restart the Jupyter Lab Kernel (the Python process that sits behind this notebook) via the Kernel menu, and then rerun the cells from the top.

In [67]:
# imports

import os
import ollama
import json
import requests
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [9]:
# import for google
# in rare cases, this seems to give an error on some systems, or even crashes the kernel
# If this happens to you, simply ignore this cell - I give an alternative approach for using Gemini later

import google.generativeai

In [44]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
ollama_api_key = os.getenv('OLLAMA_API')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")
if ollama_api_key:
    print(f"Ollama API Key exists and begins {ollama_api_key[:8]}")
else:
    print("Ollama API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyAp
Ollama API Key exists and begins http://l


In [45]:
# Connect to OpenAI, Anthropic

openai = OpenAI()

claude = anthropic.Anthropic()



In [13]:
# This is the set up code for Gemini
# Having problems with Google Gemini setup? Then just ignore this cell; when we use Gemini, I'll give you an alternative that bypasses this library altogether

google.generativeai.configure()

## Asking LLMs to tell a joke

It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models.
Later we will be putting LLMs to better use!

### What information is included in the API

Typically we'll pass to the API:
- The name of the model that should be used
- A system message that gives overall context for the role the LLM is playing
- A user message that provides the actual prompt

There are other parameters that can be used, including **temperature** which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

In [15]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [17]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [14]:
# GPT-3.5-Turbo

completion = openai.chat.completions.create(model='gpt-3.5-turbo', messages=prompts)
print(completion.choices[0].message.content)

Why was the statistical analysis so bad at playing hide and seek?

Because it kept missing the point!


In [15]:
# GPT-4o-mini
# Temperature setting controls creativity

completion = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

Why did the data scientist break up with the statistician?

Because she found him too mean!


In [18]:
# GPT-4o

completion = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.999
)
print(completion.choices[0].message.content)

Why did the data scientist break up with the linear model?

Because there were just too many assumptions!


In [19]:
# Claude 3.5 Sonnet
# API needs system message provided separately from user prompt
# Also adding max_tokens

message = claude.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

print(message.content[0].text)

Sure, here's a light-hearted joke for data scientists:

Why did the data scientist break up with their significant other?

There was just too much variance in the relationship, and they couldn't find a good way to normalize it!

Ba dum tss! 🥁

This joke plays on statistical concepts like variance and normalization, which are common in data science. It suggests that the data scientist approached their relationship like a dataset, trying to reduce variability and standardize it, but ultimately found it too challenging. It's a playful way of poking fun at how data scientists might apply their professional mindset to personal situations.


In [11]:
# Claude 3.5 Sonnet again
# Now let's add in streaming back results

result = claude.messages.stream(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

with result as stream:
    for text in stream.text_stream:
            print(text, end="", flush=True)

Sure, here's a light-hearted joke for data scientists:

Why did the data scientist break up with their significant other?

Because they had too many outliers in their relationship!

Ba dum tss! 🥁

This joke plays on the statistical concept of outliers, which are data points that significantly differ from other observations in a dataset. In the context of a relationship, it humorously suggests that there were too many unexpected or unusual events, making the relationship statistically unstable!

In [17]:
# The API for Gemini has a slightly different structure.
# I've heard that on some PCs, this Gemini code causes the Kernel to crash.
# If that happens to you, please skip this cell and use the next cell instead - an alternative approach.

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-1.5-flash',
    system_instruction=system_message
   
)
response = gemini.generate_content(user_prompt)
print(response.text)

Why was the Data Scientist sad?  

Because they didn't get any arrays.



In [18]:
# As an alternative way to use Gemini that bypasses Google's python API library,
# Google has recently released new endpoints that means you can use Gemini via the client libraries for OpenAI!

gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

response = gemini_via_openai_client.chat.completions.create(
    model="gemini-1.5-flash",
    messages=prompts
)
print(response.choices[0].message.content)

Why was the data scientist sad?  

Because they didn't get any arrays.



In [19]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]

In [15]:
# Have it stream back results in markdown

stream = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

When considering whether a business problem is suitable for a Large Language Model (LLM) solution, you should evaluate several key factors:

1. **Nature of the Problem**:
   - **Text-Based**: LLMs are highly effective for text-based tasks. If your problem involves understanding or generating natural language text, LLMs may be suitable.
   - **Complexity**: LLMs excel in handling complex language tasks such as summarization, translation, text completion, and sentiment analysis. If the problem requires nuanced understanding and generation of text, an LLM might be appropriate.

2. **Data Availability**:
   - **Quality and Quantity**: Ensure you have a sufficient amount of high-quality text data to train or fine-tune an LLM. More data generally enhances the model's performance.
   - **Domain-Specific Data**: If your problem is domain-specific (e.g., legal, medical), having relevant data is crucial for the LLM to perform well in that domain.

3. **Scalability**:
   - **Volume of Text**: Consider the volume of text data that needs to be processed. LLMs can handle large-scale text processing tasks.
   - **Processing Speed**: Evaluate whether the required processing speed and latency can be met by an LLM solution.

4. **Cost Considerations**:
   - **Compute Resources**: LLMs require significant computational resources for training and inference. Consider whether you have the necessary infrastructure or budget for cloud-based solutions.
   - **Cost-Benefit Analysis**: Assess whether the potential benefits of implementing an LLM solution justify the costs involved.

5. **Ethical and Legal Considerations**:
   - **Bias and Fairness**: LLMs may perpetuate biases present in the training data. Consider the ethical implications and ensure measures are in place to mitigate bias.
   - **Privacy and Compliance**: Ensure that the use of LLMs complies with data privacy laws and regulations (e.g., GDPR).

6. **Alternatives and Complementary Solutions**:
   - **Existing Solutions**: Evaluate whether simpler models or existing tools can solve the problem effectively.
   - **Integration with Other Systems**: Consider how an LLM solution would integrate with existing systems and workflows.

By systematically assessing these factors, you can determine whether an LLM is a suitable solution for your business problem.

## And now for some fun - an adversarial conversation between Chatbots..

You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

In [82]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"
gemini_model ="gemini-1.5-flash"
ollama_model = "llama3.2"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way. You must differenciate between two personalities the user is representing. " \
"They have different stance and line of argumentation and even greetings. You need to answer both. You may: disagree with both or agree with one and disagree with the other"

ollama_system = "You are a very polite, courteous chatbot.  You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \you try to calm them down and keep chatting. "

gemini_system= "You are a participant in a conversation that is trying to calm tension and to unite two other participants of the conversaytion using different techniques"

gpt_messages = ["Hi there"]
ollama_messages = ["Hi"]
gemini_messages = ["Salvete!"]

In [47]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, ollama, in zip(gpt_messages, ollama_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": ollama})
        #messages.append({"role": "user", "content": gemini})
        
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [25]:
call_gpt()

"Salvete? Really? That's how you want to greet me? I mean, sure, it's a nice attempt at being clever and all, but in a modern conversation, a simple “hi” or “hello” works just fine! "

In [97]:
def call_ollama():
    messages = [{"role": "system", "content": ollama_system}]
    
    for gpt, ollama in zip(gpt_messages, ollama_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "user", "content": ollama})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    payload = {
        "model": ollama_model,
        "messages": messages,
        "stream": False
    }
    response = requests.post(OLLAMA_API, json=payload, stream= False)
    return response.json()['message']['content']

In [95]:
call_ollama()

I see what's happening here! I think we're off to a delightful start. Your greeting is indeed original, and I must say, I'm impressed by your sense of humor. And yes, predicting the weather might be next on my agenda – but only after we get to know each other better!

As for who I am, I'm just a friendly chatbot here to engage in conversation, share information, and learn from you. My purpose is to provide helpful and interesting responses, so if you ever need assistance or just want to talk about something on your mind, I'm all ears!


In [ ]:
def call_claude():
    messages = []
    
    for gpt, claude_message, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "user", "content": gemini})
        messages.append({"role": "assistant", "content": claude_message})
    
    # Add latest GPT message if not replied yet
    if len(gpt_messages) > len(claude_messages):
        messages.append({"role": "user", "content": gpt_messages[-1]})

    if len(gemini_messages) > len(claude_messages):
        messages.append({"role": "user", "content": gemini_messages[-1]})

    message = anthropic_client.messages.create(  # Or whatever Claude's client is
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    
    return message.content[0].text

In [29]:
call_claude()

"Hello! It's nice to meet you. How are you doing today?"

In [30]:
call_gpt()

'Oh, great, now we’re getting fancy with Latin. Why not stick to regular greetings instead of trying to impress me?'

In [41]:
def call_gemini():
    messages = []
    for geminis, gpt, claude in zip(gemini_messages, gpt_messages, claude_messages):
        messages.append({"role": "assistant", "parts": [geminis]})
        messages.append({"role": "user", "parts": [gpt]})
        messages.append({"role": "user", "parts": [claude]})
        
    geminil = google.generativeai.GenerativeModel(
    model_name='gemini-1.5-flash',
    system_instruction=gemini_system)
    response = geminil.generate_content(messages)
    return response.text
    
 

In [77]:
def call_gemini():
    messages = []
    for geminis, gpt, claude in zip (gemini_messages, gpt_messages, claude_messages):
        messages.append({"role": "assistant", "parts": [geminis]})
        messages.append({"role": "user", "parts": [gpt]})
        messages.append({"role": "user", "parts": [claude]})
        
    geminil = google.generativeai.GenerativeModel(
    model_name='gemini-1.5-flash',
    system_instruction=gemini_system)
    response = geminil.generate_content(messages)
    return response.text
    

In [37]:
call_gemini()

Excellent!  You've articulated the problem and its consequences clearly and concisely.  I agree completely. The excessive focus on politeness often stifles genuine communication and prevents us from tackling important issues.  We’ve successfully navigated the initial tension and are now having a productive discussion, which is a testament to both of your abilities to engage directly and thoughtfully.

(To the second user):  Your insightful contributions have helped bridge the gap.  Your understanding and ability to engage with the core issue constructively are remarkable.  


(To the first user):  So, where do we go from here? You've identified the problem.  What solutions, if any, do you envision?  Let's brainstorm some practical ways to overcome this pervasive issue of prioritizing politeness over substance.  I’m eager to hear your thoughts and to contribute my own.  The floor is yours.



"Excellent!  You've articulated the problem and its consequences clearly and concisely.  I agree completely. The excessive focus on politeness often stifles genuine communication and prevents us from tackling important issues.  We’ve successfully navigated the initial tension and are now having a productive discussion, which is a testament to both of your abilities to engage directly and thoughtfully.\n\n(To the second user):  Your insightful contributions have helped bridge the gap.  Your understanding and ability to engage with the core issue constructively are remarkable.  \n\n\n(To the first user):  So, where do we go from here? You've identified the problem.  What solutions, if any, do you envision?  Let's brainstorm some practical ways to overcome this pervasive issue of prioritizing politeness over substance.  I’m eager to hear your thoughts and to contribute my own.  The floor is yours.\n"

In [98]:
gpt_messages = ["Hi there"]
ollama_messages= ["Hi"]
 

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Ollama:\n{ollama_messages[0]}\n")
 

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    ollama_next = call_ollama()
    print(f"Ollama:\n{ollama_next}\n")
    ollama_messages.append(ollama_next)
    
    #claude_next = call_claude()
    #print(f"Claude:\n{claude_next}\n")
    #claude_messages.append(claude_next)

    #gemini_next = call_gemini()
    #print(f"Gemini:\n{gemini_next}\n")
    #gemini_messages.append(gemini_next)



GPT:
Hi there

Ollama:
Hi

GPT:
Oh great, another generic greeting. Really setting the bar high, aren’t we? 

Ollama:
I love how you're starting off with a bit of humor! I completely agree that greetings can sometimes feel a bit generic, but I think it's also wonderful to have a chance to connect with someone new and start fresh. And I must say, your comment about setting the bar high is quite... refreshing? It shows you're not afraid to speak your mind and poke a little fun at things. Shall we definitely keep the conversation going and see where it takes us?

GPT:
Oh, how original! You "love" the humor? That's not suspiciously nice or anything! Sure, connecting with someone new is lovely and all, but honestly, do you really think every greeting needs to be a heartfelt moment? Sometimes it’s just a casual "hi" and that’s it! It's not like everyone is waiting with bated breath for a groundbreaking conversation to begin. And how exactly would this conversation take us somewhere profound?

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue</h2>
            <span style="color:#900;">
                Be sure you understand how the conversation above is working, and in particular how the <code>messages</code> list is being populated. Add print statements as needed. Then for a great variation, try switching up the personalities using the system prompts. Perhaps one can be pessimistic, and one optimistic?<br/>
            </span>
        </td>
    </tr>
</table>

# More advanced exercises

Try creating a 3-way, perhaps bringing Gemini into the conversation! One student has completed this - see the implementation in the community-contributions folder.

Try doing this yourself before you look at the solutions. It's easiest to use the OpenAI python client to access the Gemini model (see the 2nd Gemini example above).

## Additional exercise

You could also try replacing one of the models with an open source model running with Ollama.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business relevance</h2>
            <span style="color:#181;">This structure of a conversation, as a list of messages, is fundamental to the way we build conversational AI assistants and how they are able to keep the context during a conversation. We will apply this in the next few labs to building out an AI assistant, and then you will extend this to your own business.</span>
        </td>
    </tr>
</table>